# Signal Research Pipeline - Quick Start Demo

This notebook demonstrates the core functionality of the signal research pipeline.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Signal Research Pipeline imports
from src.data import PriceFetcher
from src.signals import MomentumSignal, MeanReversionSignal, VolatilitySignal
from src.validation.backtester import Backtester
from src.validation.metrics import PerformanceMetrics
from src.ml.feature_selection import SignalDiscovery
from src.ml.regime_detection import RegimeDetector
from src.ml.ensemble import SignalEnsemble
from src.monitoring.decay_tracker import DecayTracker, HealthScore

plt.style.use('dark_background')
print("Imports complete!")

## 1. Data Loading

In [ ]:
# Fetch crypto price data
fetcher = PriceFetcher(source='coingecko')
symbols = ['BTC', 'ETH', 'SOL', 'BNB', 'XRP']

price_df = fetcher.fetch(symbols, days=180)
print(f"Loaded {len(price_df)} rows")
price_df.head()

In [ ]:
# Visualize prices
wide_prices = fetcher.pivot_prices(price_df, 'close')
normalized = wide_prices / wide_prices.iloc[0] * 100

fig, ax = plt.subplots(figsize=(12, 6))
normalized.plot(ax=ax)
ax.set_title('Normalized Prices (Base = 100)')
ax.set_xlabel('Date')
ax.set_ylabel('Normalized Price')
plt.tight_layout()
plt.show()

## 2. Signal Generation

In [ ]:
# Create signals
momentum = MomentumSignal(lookback=20)
mean_rev = MeanReversionSignal(lookback=20)
volatility = VolatilitySignal(lookback=20)

# Compute signals
mom_signal = momentum.compute(price_df)
mr_signal = mean_rev.compute(price_df)
vol_signal = volatility.compute(price_df)

print(f"Momentum signal shape: {mom_signal.shape}")
mom_signal.head()

In [ ]:
# Visualize signal for BTC
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

for ax, (name, sig) in zip(axes, [('Momentum', mom_signal), ('Mean Reversion', mr_signal), ('Volatility', vol_signal)]):
    btc = sig[sig['symbol'] == 'BTC']
    ax.bar(btc['date'], btc['signal'], alpha=0.7, color=['green' if x > 0 else 'red' for x in btc['signal']])
    ax.axhline(0, color='white', linestyle='--', alpha=0.3)
    ax.set_title(f'{name} Signal (BTC)')
    ax.set_ylabel('Signal')

plt.tight_layout()
plt.show()

## 3. Backtesting

In [ ]:
# Run backtests
backtester = Backtester()

results = {}
for name, sig in [('momentum', mom_signal), ('mean_reversion', mr_signal), ('volatility', vol_signal)]:
    result = backtester.run(sig, price_df, signal_name=name)
    results[name] = result
    print(result)

In [ ]:
# IC comparison
fig, ax = plt.subplots(figsize=(12, 5))

for name, result in results.items():
    ic_rolling = result.ic_series.rolling(20).mean()
    ax.plot(ic_rolling.index, ic_rolling.values, label=f"{name} (IR={result.ic_ir:.2f})")

ax.axhline(0, color='white', linestyle='--', alpha=0.3)
ax.set_title('Rolling IC (20-day MA)')
ax.set_xlabel('Date')
ax.set_ylabel('IC')
ax.legend()
plt.tight_layout()
plt.show()

## 4. ML Feature Discovery

In [ ]:
# Discover important features
discovery = SignalDiscovery()
discovery_results = discovery.discover(price_df, forward_period=1, top_n=10)

print("Top discovered features:")
print(discovery_results['features'])

In [ ]:
# Feature importance plot
importance = discovery_results['importance'].head(10)

fig, ax = plt.subplots(figsize=(10, 6))
importance.plot(kind='barh', ax=ax)
ax.set_title('Feature Importance (XGBoost)')
ax.set_xlabel('Importance')
plt.tight_layout()
plt.show()

## 5. Regime Detection

In [ ]:
# Detect market regimes
regime_detector = RegimeDetector(n_regimes=3)
regime_detector.fit(price_df)

regimes = regime_detector.predict(price_df)
print("Regime labels:", regime_detector.regime_labels)
regimes.tail(10)

In [ ]:
# Regime statistics
regime_stats = regime_detector.get_regime_statistics(price_df)
print(regime_stats)

## 6. Signal Ensemble

In [ ]:
# Combine signals
signals_combined = mom_signal[['date', 'symbol']].copy()
signals_combined['momentum'] = mom_signal['signal'].values
signals_combined['mean_reversion'] = mr_signal['signal'].values
signals_combined['volatility'] = vol_signal['signal'].values

# Forward returns
returns_df = price_df.sort_values(['symbol', 'date']).copy()
returns_df['fwd_ret_1d'] = returns_df.groupby('symbol')['close'].transform(
    lambda x: x.shift(-1) / x - 1
)

# Fit ensemble
ensemble = SignalEnsemble(method='ic_weighted')
ensemble.fit(signals_combined, returns_df)

print("Ensemble weights:", ensemble.weights_)

In [ ]:
# Backtest ensemble
ensemble_signal = ensemble.combine(signals_combined)
ensemble_result = backtester.run(ensemble_signal, price_df, signal_name='ensemble')
print(ensemble_result)

## 7. Signal Monitoring

In [ ]:
# Health score for momentum signal
health_scorer = HealthScore()
health = health_scorer.calculate(results['momentum'].ic_series)

print(f"Health Score: {health['health_score']:.1f}")
print(f"Status: {health['status']}")
print("Components:", health['components'])

In [ ]:
# Decay analysis
decay_tracker = DecayTracker(rolling_window=30)

decay_info = decay_tracker.estimate_decay_rate(results['momentum'].ic_series)
print(f"Decay rate: {decay_info['decay_rate']:.6f}")
print(f"Half-life: {decay_info['half_life']:.1f} days")

## Summary

This notebook demonstrated:
1. **Data Loading** - Fetching crypto prices from CoinGecko
2. **Signal Generation** - Creating momentum, mean reversion, volatility signals
3. **Backtesting** - Walk-forward backtest with IC analysis
4. **ML Discovery** - XGBoost feature importance
5. **Regime Detection** - HMM-based market regime identification
6. **Ensemble Methods** - IC-weighted signal combination
7. **Monitoring** - Health scoring and decay analysis

Run the Streamlit dashboard for interactive exploration:
```bash
streamlit run dashboard/app.py
```